In [30]:
import pandas as pd
from datetime import date
from IPython.display import display
from random import randint, uniform

In [94]:
!pip3 install openpyxl


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\pc\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [2]:
tickers = ['PLD','AMT','PSA','SPG','DLR','EQR']
files_array = {}

for ticker in tickers:
    files_array[ticker] = pd.read_excel('yahoo_finance_reits_data.xlsx',sheet_name=ticker,index_col=0).reset_index()[-500:]

In [31]:
string = '(Open+Close+High+High)/3'
fields = ['Open','Close','High','Low','Volume']
def create_expression(string):
    for field in fields:
        string = string.replace(field,f'x[\'{field}\']')
    return string

print(create_expression(string))

(x['Open']+x['Close']+x['High']+x['High'])/3


In [32]:
def day_average(df):
    df['Day Ave'] = df.apply(lambda x: (x['Open']+x['Close']+x['High'])/3, axis=1)
    return df

In [33]:
def returns(df):
    df['Yesterday Ave'] = df['Day Ave'].shift()
    df['Dod Return'] = (df['Day Ave'] - df['Yesterday Ave'])/abs(df['Yesterday Ave'])
    df['Tommorrow Ave'] = df['Day Ave'].shift(-1)
    df['2nd Dod Return'] = (df['Tommorrow Ave'] - df['Day Ave'])/abs(df['Day Ave'])
    return df

In [34]:
def returns(df):
    df['Yesterday Ave'] = df['Day Ave'].shift()
    df['Dod Return'] = (df['Day Ave'] - df['Yesterday Ave'])/abs(df['Yesterday Ave'])
    df['Tommorrow Ave'] = df['Day Ave'].shift(-1)
    df['2nd Dod Return'] = (df['Tommorrow Ave'] - df['Day Ave'])/abs(df['Day Ave'])
    return df

In [35]:
def alpha_value(df,string):
    df['Alpha Value'] = df.apply(lambda x: eval(create_expression(string)), axis=1)
    return df

In [36]:
def calculate_alpha_performance(files_array,alpha_expr):
    print(f"Calculating performance for alpha: {alpha_expr}")
    combi = pd.DataFrame(files_array[next(iter(files_array))].reset_index()['Date']) # get list of dates from any ticker
    for key,value in files_array.items():
        day_average(value)
        returns(value)
        dod_return = pd.DataFrame(value[['Date','Dod Return']])
        dod_return.rename(columns={'Dod Return':f'{key} Dod Return'},inplace=True)
        combi = combi.merge(dod_return,on='Date',how='left')
    #print("\nDay on day returns for each ticker:")
    #print(files_array)
    #display(combi)
    
    combination = combi.melt(['Date'], var_name='Ticker', value_name='Dod Return') #need to double melt 2nd dod return
    combination.sort_values('Date', inplace=True)
    combination.dropna(inplace=True)
    #print("\nTranspose df:")
    #display(combination)
    
    combi2 = pd.DataFrame(files_array[next(iter(files_array))].reset_index()['Date'])
    for key,value in files_array.items():
        day_average(value)
        returns(value)
        dod_return = pd.DataFrame(value[['Date','2nd Dod Return']])
        dod_return.rename(columns={'2nd Dod Return':f'{key} Dod Return'},inplace=True)
        combi2 = combi2.merge(dod_return,on='Date',how='left')
    
    #print("\nDay on day returns for following day for each ticker:")
    #display(combi2)
    
    combination2 = combi2.melt(['Date'], var_name='Ticker', value_name='2nd Dod Return') #need to double melt 2nd dod return
    combination2.sort_values('Date', inplace=True)
    combination2.dropna(inplace=True)
    ##print("\nTranspose df for following day:")
    #display(combination2)
    
    combi3 = pd.DataFrame(files_array[next(iter(files_array))].reset_index()['Date']) # get list of dates from any ticker
    for key,value in files_array.items():
        alpha_value(value,alpha_expr)
        alpha_return = pd.DataFrame(value[['Date','Alpha Value']])
        alpha_return.rename(columns={'Alpha Value':f'{key} Dod Return'},inplace=True)
        #print("TESTTTT")
        #display(alpha_return)
        #display(combi)
        combi3 = combi3.merge(alpha_return,on='Date',how='left')
    #print("COMBI 3")
    #display(combi3)
        
    #print("\nAlpha value for each ticker:")
    combination4 = combi3.melt(['Date'], var_name='Ticker', value_name='Alpha Value') #need to double melt 2nd dod return
    combination4.sort_values('Date', inplace=True)
    combination4.dropna(inplace=True)
    #print("COMBI 4")
    #display(combination4)
    
    
    combination3 = combination.merge(combination2,how='left',on=['Date','Ticker'])
    combination3 = combination3.merge(combination4,how='left',on=['Date','Ticker'])
    #print("\nMerge both dfs:")
    #display(combination3)
    
    combination3.dropna(inplace=True)
    combination3['Rank'] = combination3.sort_values(['Alpha Value'], ascending=[True]).groupby(['Date']).cumcount() + 1
    combination3['Alpha Rank'] = combination3['Rank'].apply(lambda x: x * (1/(len(tickers)-1)) - (1/(len(tickers)-1)))
    combination3['Centered 0'] = combination3['Alpha Rank'] - 0.5
    combination3['Abs Centered 0'] = abs(combination3['Centered 0'])
    combination3['Sum Rank'] = combination3.apply(lambda x: sum(combination3[combination3['Date']==x['Date']]['Abs Centered 0']),axis=1)
    combination3['Normalized'] = combination3.apply(lambda x: -x['Abs Centered 0']/x['Sum Rank'] if x['Centered 0'] < 0 else x['Abs Centered 0']/x['Sum Rank'], axis=1)
    combination3['Allocated'] = combination3['Normalized'] * 20
    combination3['PnL'] = combination3['Allocated'] * combination3['2nd Dod Return']
    #print("\nAllocate portfolio based on Alpha Rank:")
    #display(combination3)
    
    final = combination3[['Date','PnL']]
    pnl = final.groupby(['Date']).sum()
    pnl['PnL'] = pnl['PnL'].cumsum()
    pnl.reset_index(inplace=True)
    #display(pnl)
    #pnl.plot(kind='line',x='Date',y='PnL');
    
    return combination3,pnl
    

In [37]:
resultDf,pnlDf = calculate_alpha_performance(files_array,string)

Calculating performance for alpha: (Open+Close+High+High)/3


In [38]:
def objective(files_array,candidate_string):
    # Evaluate profit for alpha candidate for 1 year period
    resultDf,pnlDf = calculate_alpha_performance(files_array,candidate_string)
    one_year_return = pnlDf.iloc[-6,1] - pnlDf.iloc[-365,1]  
    six_month_return = pnlDf.iloc[-6,1] - pnlDf.iloc[-180,1]
    three_month_return = pnlDf.iloc[-6,1] - pnlDf.iloc[-90,1]  
    one_month_return = pnlDf.iloc[-6,1] - pnlDf.iloc[-30,1]  
    average_return = (one_month_return + three_month_return + six_month_return + one_year_return)/4
    print("AVERAGE RETURN: ", average_return)
    return average_return

In [39]:
objective(files_array,r"(Close - (High - Close / (High) / Open))")

Calculating performance for alpha: (Close - (High - Close / (High) / Open))
AVERAGE RETURN:  9.445009759709972


9.445009759709972

In [40]:
n_pop = 10
n_bits = 8
n_iter = 5
r_cross = 0.8 # crossover rate
r_mut = 0.1 # flip bits with probability

In [41]:
p1 = '(Open + Volume + (Close) - Low / ((Low + Close)))'
p2 = 'Close - (High * High - Low + Open * High)'
r_cross = 0.8

def crossover(p1,p2,r_cross):
    operators = ['+','-','/','*']
    
    if uniform(0,1) < r_cross:
        #p1
        valid_crossover_p1 = []
        for idx in range(len(p1)):
            char = p1[idx]
            if char in operators:
                valid_crossover_p1.append(idx)
        print(valid_crossover_p1)
        chosen_idx_p1 = valid_crossover_p1[randint(0,len(valid_crossover_p1)-1)]         
        left_p1 = p1[:chosen_idx_p1-1]
        right_p1 = p1[chosen_idx_p1+2:]
        fixed_left_p1 = left_p1 + find_missing_bracket(left_p1)
        fixed_right_p1 = find_missing_bracket(right_p1) + right_p1
        print("fixed left p1: ",fixed_left_p1," fixed right p1: ",fixed_right_p1)

        #p2
        valid_crossover_p2 = []
        for idx in range(len(p2)):
            char = p2[idx]
            if char in operators:
                valid_crossover_p2.append(idx)
        print(valid_crossover_p2)
        chosen_idx_p2 = valid_crossover_p2[randint(0,len(valid_crossover_p2)-1)]         
        left_p2 = p2[:chosen_idx_p2-1]
        right_p2 = p2[chosen_idx_p2+2:]
        fixed_left_p2 = left_p2 + find_missing_bracket(left_p2)
        fixed_right_p2 = find_missing_bracket(right_p2) + right_p2
        print("fixed left p2: ",fixed_left_p2," fixed right p2: ",fixed_right_p2)
        
        c1 = fixed_left_p1 + ' + ' + fixed_right_p2
        c2 = fixed_left_p2 + ' + ' + fixed_right_p1
        return [c1,c2]
    return [p1,p2]

In [42]:
def genetic_algorithm(objective,files_array, n_bits, n_iter, n_pop, r_cross, r_mut,fields):
    # initial population of random bitstring
    pop = generateAlphas(n_pop)
    print(pop)
    # keep track of best solution
    best, best_eval = 0, objective(files_array,pop[0])
    # enumerate generations
    for gen in range(n_iter):
        # evaluate all candidates in the population
        scores = [objective(files_array,c) for c in pop]
        print(scores)
        # check for new best solution
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen, pop[i], scores[i]))
                
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        print("Selected Parents",selected)
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
        # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, fields, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

In [43]:
def selection(pop,scores,k=3):
    # first random selection
    selection_ix = randint(0,len(pop)-1)
    for ix in [randint(0,len(pop)-1) for x in range(k-1)]:
    # check if better (e.g. perform a tournament)
        if scores[ix] > scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

In [44]:
r_mut = 0.1
def mutation(string,fields,r_mut):
    for feature in fields:
        print("STRING",string)
        print(feature)
        split = string.split(feature)
        print("split: ",split)
        terms = len(split)
        if terms > 1:
            replacements = []
            for i in range(terms-1):
                if uniform(0,1) < r_mut:
                    replacements.append(fields[randint(0,len(fields)-1)])
                else:
                    replacements.append(feature)
            print("replacements: ",replacements)            
        string = ''
        for i in range(len(split)):
            string += split[i]
            if i < (len(split)-1):
                string += replacements[i]


In [45]:
from random import random, randint, choice

def randomAlpha(prob,datafields):
    p = random()
    if p > prob:
        return datafields[randint(0, 4)]
    elif randint(0, 1) == 0:
        return ParenthesizedExpression(randomAlpha(prob / 1.2,datafields))
    else:
        left = randomAlpha(prob / 1.2,datafields)
        op = choice(["+", "-", "*", "/"])
        right = randomAlpha(prob / 1.2,datafields)
        return BinaryExpression(left, op, right)
        #return str(left) + ' ' + str(op) + ' ' + str(right) 

def generateAlphas(number):
    past_alphas = []
    alphas_to_sim = []
    for i in range(number):
        duplicate = True
        while duplicate:
            generated = randomAlpha(1,fields)
            if generated not in past_alphas:
                duplicate = False
                #past_alphas.append(generated) # use this to output expressions
                past_alphas.append(str(generated)) # use this to output string
    return past_alphas


In [46]:
# Possible additions include methods to evaluate expressions and generate
# some random expressions.

class Expression:
    pass

class Number(Expression):
    def __init__(self, num):
        self.num = num

    def __str__(self):
        return str(self.num)

class BinaryExpression(Expression):
    def __init__(self, left, op, right):
        self.left = left
        self.op = op
        self.right = right

    def __str__(self):
        return str(self.left) + " " + self.op + " "  + str(self.right)

class ParenthesizedExpression(Expression):
    def __init__(self, exp):
        self.exp = exp

    def __str__(self):
        return "(" + str(self.exp) + ")"

e1 = Number(5)

In [48]:
def find_missing_bracket(input_string):
    # return the number of missing brackets
    close_bracket_count = 0
    open_bracket_count = 0
    for char in input_string:
        if char == '(':
            open_bracket_count += 1
        elif char == ')':
            close_bracket_count += 1
    abs_difference = abs(open_bracket_count-close_bracket_count) 
    if open_bracket_count > close_bracket_count:
        return abs_difference*")"
    elif close_bracket_count > open_bracket_count:
        return abs_difference*"("
    else:
        return ""

In [49]:
genetic_algorithm(objective, files_array, n_bits, n_iter, n_pop, r_cross, r_mut, fields)

['(High)', '(Open) - ((Low))', '(Open + ((High)))', '(((Volume) / Close - Volume * Low)) + (Close) * (Close) - Low / Close - Open / Volume', 'Open - (((Close)) + Low) / ((High)) / Volume', '(Low) + High * Close - Volume * Close - (Close - Open + Close - Close * Open) / High', 'High / Open / (Open) - Open', '((Low) - Close) + Volume - (Low * High / ((High))) + Open', '((Low) - ((Close - Volume)) * Open)', '((((Close))))']
Calculating performance for alpha: (High)
AVERAGE RETURN:  -1.1719654953510275
Calculating performance for alpha: (High)
AVERAGE RETURN:  -1.1719654953510275
Calculating performance for alpha: (Open) - ((Low))
AVERAGE RETURN:  -8.850980312857143
Calculating performance for alpha: (Open + ((High)))
AVERAGE RETURN:  -1.3127869414782078
Calculating performance for alpha: (((Volume) / Close - Volume * Low)) + (Close) * (Close) - Low / Close - Open / Volume
AVERAGE RETURN:  0.860689706373285
Calculating performance for alpha: Open - (((Close)) + Low) / ((High)) / Volume
AVE

AVERAGE RETURN:  -0.6108777094078656
Calculating performance for alpha: (Low) + High * Close - Volume * Close + (((Close - Volume)) * Open)
AVERAGE RETURN:  0.8531794654588838
Calculating performance for alpha: High + (((Close - Volume)) * Open)
AVERAGE RETURN:  1.013796188321825
Calculating performance for alpha: ((Low)) + Open / (Open) - Open
AVERAGE RETURN:  8.880583934374993
Calculating performance for alpha: ((Low) - ((Close - Volume)) * Open)
AVERAGE RETURN:  -1.013796188321825
Calculating performance for alpha: ((Low) - ((Close - Volume)) * Open)
AVERAGE RETURN:  -1.013796188321825
Calculating performance for alpha: (((Volume) / Close - Volume * Low)) + (Close) * (Close) + (((Volume)) * Open)
AVERAGE RETURN:  -7.3005063039584615
Calculating performance for alpha: ((Low) - ((Close))) + Low / Close - Open / Volume
AVERAGE RETURN:  -7.23346806893505
Calculating performance for alpha: (((Volume) / Close - Volume * Low)) + (Close) + (((Close - Volume)) * Open)
AVERAGE RETURN:  1.0001

AVERAGE RETURN:  0.8531794654588838
Calculating performance for alpha: ((Low)) + Open / (Open) - Open
AVERAGE RETURN:  8.880583934374993
Calculating performance for alpha: ((Low)) + Open / (Open) - Open
AVERAGE RETURN:  8.880583934374993
Calculating performance for alpha: ((Low)) + Open / (Open) - Open
AVERAGE RETURN:  8.880583934374993
Calculating performance for alpha: ((Low)) + Open / (Open) - Open
AVERAGE RETURN:  8.880583934374993
Calculating performance for alpha: ((Low)) + (Close - Open + Close - Close * Open) / High
AVERAGE RETURN:  -0.6108777094078656
Calculating performance for alpha: High + (((Close - Volume)) * Open)
AVERAGE RETURN:  1.013796188321825
Calculating performance for alpha: ((Low)) + (Close - Open + Close - Close * Open) / High
AVERAGE RETURN:  -0.6108777094078656
Calculating performance for alpha: (((Volume) / Close - Volume * Low)) + (Close) + (((Close - Volume))) + Open
AVERAGE RETURN:  0.8719303262546727
Calculating performance for alpha: ((Low)) + Open / (O

AVERAGE RETURN:  -0.6108777094078656
Calculating performance for alpha: ((Low)) + Open / (Open) - Open
AVERAGE RETURN:  8.880583934374993
Calculating performance for alpha: (Low) + High * Close - Volume * Close + (((Close - Volume)) * Open)
AVERAGE RETURN:  0.8531794654588838
Calculating performance for alpha: ((Low)) + Open / (Open) - Open
AVERAGE RETURN:  8.880583934374993
Calculating performance for alpha: ((Low)) + (Close) + (Open) - Open
AVERAGE RETURN:  -1.0662776416991875
Calculating performance for alpha: ((Low)) + Open + (Open + Close - Close * Open) / High
AVERAGE RETURN:  -1.3911260227932924
Calculating performance for alpha: ((Low)) + Open / (Open) + (Open) - Open
AVERAGE RETURN:  -1.0919169958341115
Calculating performance for alpha: ((Low)) + Open + Open
AVERAGE RETURN:  -1.303977625826747
Calculating performance for alpha: (((Volume) / Close - Volume)) + (Open) - Open
AVERAGE RETURN:  -0.01776223025161111
Calculating performance for alpha: ((Low)) + Open + ((Low)) + (Clo

AVERAGE RETURN:  -1.303977625826747
Calculating performance for alpha: (((Volume) / Close - Volume)) + (Open) + (Open) - Open
AVERAGE RETURN:  -0.01776223025161111
Calculating performance for alpha: ((Low)) + Open / (Open) + Open
AVERAGE RETURN:  -1.1634112384665543
Calculating performance for alpha: ((Low)) + Open / (Open) + Open
AVERAGE RETURN:  -1.1634112384665543
Calculating performance for alpha: ((Low)) + Open / (Open) - Open
AVERAGE RETURN:  8.880583934374993
Calculating performance for alpha: ((Low)) + Open / (Open) - Open
AVERAGE RETURN:  8.880583934374993
Calculating performance for alpha: ((Low)) + Open / (Open) + (Open) - Open
AVERAGE RETURN:  -1.0919169958341115
Calculating performance for alpha: ((Low)) + Open + ((Low)) + (Close) + (((Close - Volume))) + Open
AVERAGE RETURN:  0.0739155667408849
[8.880583934374993, 8.880583934374993, -1.303977625826747, -0.01776223025161111, -1.1634112384665543, -1.1634112384665543, 8.880583934374993, 8.880583934374993, -1.0919169958341115

['((Low)) + Open / (Open) - Open', 8.880583934374993]

In [92]:
generateAlphas(10)

['((Low / ((Volume)) * Volume)) / (((Low)) / Low / (Open))',
 '(Open / Volume * Volume)',
 'Volume + High + Low - ((Volume))',
 'High * ((Open))',
 'Volume - Low',
 '((Volume))',
 '(Open + (Open))',
 '(Close) * Close + Low * Low - Volume * Close',
 '(Low)',
 '(Open + (High) / Low)']

In [93]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = []

def parse_expression(tokens):
    node = parse_term(tokens)

    while tokens and tokens[0] in ['+', '-']:
        operator = tokens.pop(0)
        right = parse_term(tokens)
        new_node = Node(operator)
        new_node.children.extend([node, right])
        node = new_node

    return node

def parse_term(tokens):
    node = parse_factor(tokens)

    while tokens and tokens[0] in ['*', '/']:
        operator = tokens.pop(0)
        right = parse_factor(tokens)
        new_node = Node(operator)
        new_node.children.extend([node, right])
        node = new_node

    return node

def parse_factor(tokens):
    if not tokens:
        raise ValueError("Unexpected end of expression")

    token = tokens.pop(0)

    if token.isalnum():  # Changed from isdigit() to isalnum() to handle alphabetic tokens
        return Node(token)
    elif token == '(':
        node = parse_expression(tokens)
        if tokens[0] != ')':
            raise ValueError("Expected ')'")
        tokens.pop(0)  # Consume ')'
        return node
    else:
        raise ValueError(f"Unexpected token: {token}")

def parse_math_expression(expression):
    tokens = expression.replace('(', ' ( ').replace(')', ' ) ').split()
    return parse_expression(tokens)

# Example usage
math_expression = "((Open / Volume) + Open - Open * (Volume) / Volume * ((Close - Open)))"
parse_tree_root = parse_math_expression(math_expression)

math_expression = "((Open / Volume) + Open - Open * (Volume) / Volume * ((Close - Open)))"
p1 = parse_math_expression(math_expression)

math_expression = "(Open - Open * (Volume) / Volume * ((Close - Open)))"
p2 = parse_math_expression(math_expression)

# Helper function to visualize the tree
def print_tree(node, depth=0, prefix=""):
    # Handle the root node
    if depth == 0:
        print(node.value)
    else:
        print(prefix + "└── " + node.value)

    if node.children:
        new_prefix = prefix + ("    " if depth == 0 else "│   ")
        for i, child in enumerate(node.children):
            # Update the prefix for the last child
            if i == len(node.children) - 1:
                new_prefix = prefix + "    "
            print_tree(child, depth + 1, new_prefix)

print_tree(parse_tree_root)

-
    └── +
    │   └── /
    │   │   └── Open
    │       └── Volume
        └── Open
    └── *
    │   └── /
    │   │   └── *
    │   │   │   └── Open
    │   │       └── Volume
    │       └── Volume
        └── -
        │   └── Close
            └── Open


In [94]:
def tree_to_expression(node):
    # If the node is a leaf (has no children), return its value
    if not node.children:
        return node.value

    # If the node is not a leaf, it should be an operator
    # We then recursively get the expressions of its children
    left_expr = tree_to_expression(node.children[0])
    right_expr = tree_to_expression(node.children[1])

    # Construct and return the expression string
    return f"({left_expr} {node.value} {right_expr})"

# Example usage
# Assuming root is the root of your expression tree
expression = tree_to_expression(p1)
print(expression)

(((Open / Volume) + Open) - (((Open * Volume) / Volume) * (Close - Open)))


In [50]:
objective(files_array,'((Low)) + Open / (Open) - Open')

Calculating performance for alpha: ((Low)) + Open / (Open) - Open
AVERAGE RETURN:  8.880583934374993


8.880583934374993

In [97]:
print(crossover(p1,p2,r_cross))

fixed left p1:  (Open + Volume)  fixed right p1:  ((Close) - Low / ((Low + Close)))
fixed left p2:  Close - (High * High - Low + Open)  fixed right p2:  (High)
['(Open + Volume) + (High)', 'Close - (High * High - Low + Open) + ((Close) - Low / ((Low + Close)))']


In [41]:
crossover(p1,p2,r_cross)

fixed left p1:  (Open + Volume + (Close) - Low / ((Low)))  fixed right p1:  (((Close)))
fixed left p2:  Close - (High * High - Low)  fixed right p2:  (Open * High)


In [37]:
find_missing_bracket('(Open + Volume + (Close)')

')'